In [32]:
from sklearn.metrics import make_scorer, accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import pandas as pd
from sklearn.base import clone
import matplotlib.pyplot as plt

from mlflow.models import infer_signature
from urllib.parse import urlparse
import logging
import mlflow

## Model Hyperparameter

In [33]:
models = [
    {
        'model': LogisticRegression(),
        'name': 'Logistic Regression',
        'param_grid': {
            'penalty': ['l1', 'l2', 'elasticnet', None],
            'C': [0.01, 0.1, 1, 10, 100],
            'solver': ['saga', 'liblinear'],
            'max_iter': [100, 200, 500]
        }
    },
    {
        'model': SVC(probability=True),
        'name': 'Support Vector Classifier',
        'param_grid': dict()
    },
    {
        'model': RandomForestClassifier(),
        'name': 'Random Forest Classifier',
        'param_grid': {
            'n_estimators': [100, 200, 500],
            'max_depth': [None, 5, 10, 20],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'bootstrap': [True, False]
        }
    }
]

In [34]:
datasets = [
    ['Label Encoding', pd.read_csv('../data/processed/data_v1.csv')],
    ['One-hot Encoding', pd.read_csv('../data/processed/data_v2.csv')],
    ['Label Encoding + MinMax Scaler', pd.read_csv('../data/processed/data_v3.csv')],
    ['Label Encoding + Standard Scaler', pd.read_csv('../data/processed/data_v4.csv')],
    ['One-hot Encoding + MinMax Scaler', pd.read_csv('../data/processed/data_v5.csv')],
    ['One-hot Encoding + Standard Scaler', pd.read_csv('../data/processed/data_v6.csv')]
]

In [35]:
# Initialize MLflow
mlflow.set_experiment("Tuberculosis Prediction")
mlflow.set_tracking_uri("http://localhost:5000")
tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme

In [36]:
with mlflow.start_run(run_name="Tuberculosis Prediction Model Training", nested=False):
    for i in datasets:
        dataset_name = i[0]
        X = i[-1].drop(columns=['Class'])
        y = i[-1]['Class']
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

        for j in models:
            run_name=f"{i[0]} + {j['name']}"
            logging.info(f"Training {run_name}...")
            base_model = clone(j['model'])
            cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            model = RandomizedSearchCV(base_model, j['param_grid'], cv=cv, random_state=42, scoring='accuracy', refit='accuracy')
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)

            signature=infer_signature(X_train,y_train)
            with mlflow.start_run(run_name=run_name, nested=True):
                mlflow.log_param("model", run_name)
                mlflow.log_params(model.best_params_)
                mlflow.log_metric('accuracy', accuracy)
                cm=confusion_matrix(y_test,y_pred)
                cr=classification_report(y_test,y_pred)

                disp = ConfusionMatrixDisplay(confusion_matrix=cm)
                fig, ax = plt.subplots(figsize=(6, 6))
                disp.plot(ax=ax)
                plt.title("Confusion Matrix")

                # Save & log the figure to MLflow
                mlflow.log_figure(fig, "confusion_matrix_plot.png")

                # Close the plot to avoid memory issues in loops
                plt.close(fig)

                mlflow.log_text(str(cm),"confusion_matrix.txt")
                mlflow.log_text(cr,"classification_report.txt")
                # mlflow.sklearn.log_model(model, "model")

                if tracking_url_type_store !='file':
                    mlflow.sklearn.log_model(model,"model",registered_model_name=f"Best {run_name}")
                else:
                    mlflow.sklearn.log_model(model,"model",signature=signature)

d:\My\AboutData\Project\Tuberculosis-Prediction\tbc_prediction_venv\lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\My\AboutData\Project\Tuberculosis-Prediction\tbc_prediction_venv\lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\My\AboutData\Project\Tuberculosis-Prediction\tbc_prediction_venv\lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\My\AboutData\Project\Tuberculosis-Prediction\tbc_prediction_venv\lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
d:\My\AboutData\Project\Tuberculosis-Prediction\tbc_prediction_venv\lib\site-packages\sklearn\linear_model\_sag.py:3

🏃 View run Label Encoding + Logistic Regression at: http://localhost:5000/#/experiments/489304826085007307/runs/375d093eee074c648b6fbabb0abd2249
🧪 View experiment at: http://localhost:5000/#/experiments/489304826085007307


d:\My\AboutData\Project\Tuberculosis-Prediction\tbc_prediction_venv\lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
d:\My\AboutData\Project\Tuberculosis-Prediction\tbc_prediction_venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in la

🏃 View run Label Encoding + Support Vector Classifier at: http://localhost:5000/#/experiments/489304826085007307/runs/8a61aac8107d4f56984a76ebd596e22d
🧪 View experiment at: http://localhost:5000/#/experiments/489304826085007307


d:\My\AboutData\Project\Tuberculosis-Prediction\tbc_prediction_venv\lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
d:\My\AboutData\Project\Tuberculosis-Prediction\tbc_prediction_venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in la

...

🏃 View run One-hot Encoding + Standard Scaler + Logistic Regression at: http://localhost:5000/#/experiments/489304826085007307/runs/63ef4da510c44e839f5a171780997f36
🧪 View experiment at: http://localhost:5000/#/experiments/489304826085007307


d:\My\AboutData\Project\Tuberculosis-Prediction\tbc_prediction_venv\lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
d:\My\AboutData\Project\Tuberculosis-Prediction\tbc_prediction_venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in la

🏃 View run One-hot Encoding + Standard Scaler + Support Vector Classifier at: http://localhost:5000/#/experiments/489304826085007307/runs/631ff5dd16d54dfbbaa19dc339a52661
🧪 View experiment at: http://localhost:5000/#/experiments/489304826085007307


d:\My\AboutData\Project\Tuberculosis-Prediction\tbc_prediction_venv\lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
d:\My\AboutData\Project\Tuberculosis-Prediction\tbc_prediction_venv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in la

🏃 View run One-hot Encoding + Standard Scaler + Random Forest Classifier at: http://localhost:5000/#/experiments/489304826085007307/runs/a35a3c8edf6744ffb29679fc0346ac9e
🧪 View experiment at: http://localhost:5000/#/experiments/489304826085007307
🏃 View run Tuberculosis Prediction Model Training at: http://localhost:5000/#/experiments/489304826085007307/runs/d47ec60061ea4f3381cb1dd6a45ce9f5
🧪 View experiment at: http://localhost:5000/#/experiments/489304826085007307


0.6965